In [7]:
from mpi4py import MPI
import coqui

# CoQui computing environemnt: mpi handler and verbosity
mpi = coqui.MpiHandler()
coqui.set_verbosity(mpi, output_level=4, debug_level=0)
if mpi.root():
    print(mpi)

CoQuí MPI state
-----------------
  Global communicator: rank 1 / 1
  Internode communicator: rank 1 / 1
  Intranode communicator: rank 1 / 1


## ⚛️ First, defining Your System!

Every CoQuí calculation begins with the declaration of a *target* system, which specifies all the ingredients required to construct the non-interacting Hamiltonian:  

$$
(H_{0})^{\textbf{k}}_{ab} = \int d\textbf{r} \phi^{\textbf{k}*}_{a}(\textbf{r}) \Big [ \frac{\nabla^{2}}{2} +  V_{\mathrm{ext}}(\textbf{r}) \big] \phi^{\textbf{k}}_{b}(\textbf{r})
$$

This setup requires the following components:

- **Metadata**: Basic information describing the system (e.g., structure, pseudopotentials, energy cutoffs, etc.)
- **Single-Particle Basis Functions $\phi^{\textbf{k}}_{a}(\textbf{r})$**:
  The basis used to expand the many-body Hamiltonian. This can include Kohn–Sham orbitals, Gaussian-type orbitals, or other grid-based representations. 
- **Mean-Field Solution** (optional): DFT or HF solution used as a starting point for correlated many-body methods. 

---
Specifically, in this tutorial, you will learn how to use CoQuí:

- Initialize a system handler in CoQuí, which wraps all the data required for post-mean-field simulations.

## 🧱 `Mf` class 

In CoQuí, the declaration of a simulated system is handled by the `Mf` class, which encapsulates all the necessary data described in the previous section. 

A standard usage looks like:
```python
# Mf for the target system
mf_params = {
    "prefix": "si", 
    "outdir": "data/qe_inputs/si/out"
}
mf = coqui.make_mf(mpi, mf_params, "qe")
```
where the `Mf` object is constructed via: 

```python
coqui.make_mf(mpi: coqui.MpiHandler, mf_params: dict, mf_type: str) -> coqui.Mf
```

This functions requires 
- `mpi`: an `MpiHandler` object that manages MPI context
- `mf_params`: a dictionary of parameters used during the construction of `Mf`.
- `mf_type`: a string specfying the type of DFT backend. Currently, CoQuí supports: 
  -  [Quantum ESPRESSO](https://www.quantum-espresso.org): `mf_type = "qe"`
  -  [PySCF](https://pyscf.org): `mf_type = "pyscf"` 

Once initialized, this class is **read-only** and serves as the interface between CoQuí and the output of supported DFT codes, providing access to:
  1. **System metadata** $-$ such as the number of bands, spins, and k-points.
  2. **Single-particle basis functions** $-$ used to expand a many-body Hamiltonian


> 💡 **NOTE** 💡
> - Every DFT package produces its output in a unique format. Under the hood, different backend-specific readers are called, selected via the mf_type argument.
> - Once the `Mf` object is created, all subsequent CoQuí routines become agnostic to the origin of the mean-field data, allowing a unified interface for downstream many-body methods. 

### 🧪 Exercise 1: Initialize your first mean-field handler

- Copy and paste the above python script
- Modify `mf_params` to read the Qunatum Espresso output stored in `data/xxx/out`. 
- How many k-points in the full BZ and irreducible BZ respectively? 

> 💡 **HINT** 💡
> - A full list of parameters for `mf_params` can be viewed using `help(coqui.make_mf)`. 
> - Check the QE input (`data/xxx/prefix.nscf.in`) to set `mf_param` accordingly                                                                     

In [8]:
# construct a mean-field handler by reading an existing QE calculation 
mf_params = {
    "prefix": "si", 
    "outdir": "../data/qe_inputs/si_222/out", 
    "filetype": "h5"
}
my_mf = coqui.make_mf(mpi, mf_params, "qe")

In [6]:
print(my_mf)

CoQuí mean-field state
----------------------
  Type                : qe
  Prefix              : si
  Output dir          : ../data/qe_inputs/si_222/out
  Number of electrons (nelec): 8
  Bands (nbnd)        : 80
  Spins (nspin)       : 1
  Polarization (npol) : 1
  Monkhorst-Pack grid : (2, 2, 2)
  K-points            : 8 total, 4 in IBZ
  Q-points            : 8 total, 4 in IBZ
  Kinetic energy cutoff (ecutrho): 200 a.u.
  FFT grid            : (36, 36, 36)
